In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import re
import pandas as pd
import json
from collections import defaultdict
from collections import OrderedDict
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pickle

In [ ]:
#하드부팅 데이터
soft_data_1 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/양병진_softvoting/pred_data_5pt.json', 'r')]
soft_data_2 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/양병진_softvoting/pred_data_5pt_1cha.json', 'r')]
soft_data_3 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/양병진_softvoting/pred_data_5pt_2cha.json', 'r')]


hard_data_1 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/최종본/김규인스페셜/0.6210_6167_618_618기준_멀티살림_12개하드부팅_멀티살림.json', 'r')]
hard_data_2 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/상위점수*상위공백/3개하드보팅/3de/highscore_b_1_3_2.json', 'r')]
hard_data_3 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/H.O.F/상위점수*상위공백/3개하드보팅/3ac/highscore_b_3_1_2.json', 'r')]


dan_data_1 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/data_10_17/pred/pred_21_12.json', 'r')]
dan_data_2 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/data_train_dev_label23/pred/pred_23_5_12.json', 'r')]
dan_data_3 = [json.loads(line) for line in open('/content/drive/MyDrive/메인프로젝트(5조)/말뭉치 경진대회/data_10_17/pred/pred_5_12.json', 'r')]


In [ ]:
#결과값 저장할 제이슨 형식
data_final = [json.loads(line) for line in open('/content/drive/MyDrive/소장용 알파코 프로젝트/메인프로젝트/bestepochpt/hard_voting/dan/pred_5_12.json', 'r')]

In [ ]:
#하드부팅 가중치 설정

wv_0 = 62.1
wv_1 = 61.67
wv_2 = 52.25
wv_3 = 53.35
wv_4 = 59.44
wv_5 = 59.7
wv_6 = 60.11
wv_7 = 1
wv_8 = 2

In [ ]:
# 1 12 12 - 1 - 2 - 3
# 1 12 13 - 1 - 2 - 3
# 1 12 23 - 1 - 2 - 3
# 1 13 12 - 1 - 2 - 3
# 1 13 13 - 1 - 2 - 3
# 1 13 23 - 1 - 2 - 3
# 1 23 12 - 1 - 2 - 3
# 1 23 13 - 1 - 2 - 3
# 1 23 23 - 1 - 2 - 3

In [ ]:
score = [
    [soft_data_1,hard_voting_1, hard_voting_2, dan_1, dan_2], # 1 12 12
    [soft_data_1,hard_voting_1, hard_voting_2, dan_1, dan_3], # 1 12 13
    [soft_data_1,hard_voting_1, hard_voting_2, dan_2, dan_3], # 1 12 23
    [soft_data_1,hard_voting_1, hard_voting_3, dan_1, dan_2], # 1 13 12
    [soft_data_1,hard_voting_1, hard_voting_3, dan_1, dan_3], # 1 13 13
    [soft_data_1,hard_voting_1, hard_voting_3, dan_2, dan_3], # 1 13 23
    [soft_data_1,hard_voting_2, hard_voting_3, dan_1, dan_2], # 1 23 12
    [soft_data_1,hard_voting_2, hard_voting_3, dan_1, dan_3], # 1 23 13
    [soft_data_1,hard_voting_2, hard_voting_3, dan_2, dan_3], # 1 23 23
    #
    [soft_data_2,hard_voting_1, hard_voting_2, dan_1, dan_2], # 2 12 12
    [soft_data_2,hard_voting_1, hard_voting_2, dan_1, dan_3], # 2 12 13
    [soft_data_2,hard_voting_1, hard_voting_2, dan_2, dan_3], # 2 12 23
    [soft_data_2,hard_voting_1, hard_voting_3, dan_1, dan_2], # 2 13 12
    [soft_data_2,hard_voting_1, hard_voting_3, dan_1, dan_3], # 2 13 13
    [soft_data_2,hard_voting_1, hard_voting_3, dan_2, dan_3], # 2 13 23
    [soft_data_2,hard_voting_2, hard_voting_3, dan_1, dan_2], # 2 23 12
    [soft_data_2,hard_voting_2, hard_voting_3, dan_1, dan_3], # 2 23 13
    [soft_data_2,hard_voting_2, hard_voting_3, dan_2, dan_3], # 2 23 23
    #
    [soft_data_3,hard_voting_1, hard_voting_2, dan_1, dan_2], # 3 12 12
    [soft_data_3,hard_voting_1, hard_voting_2, dan_1, dan_3], # 3 12 13
    [soft_data_3,hard_voting_1, hard_voting_2, dan_2, dan_3], # 3 12 23
    [soft_data_3,hard_voting_1, hard_voting_3, dan_1, dan_2], # 3 13 12
    [soft_data_3,hard_voting_1, hard_voting_3, dan_1, dan_3], # 3 13 13
    [soft_data_3,hard_voting_1, hard_voting_3, dan_2, dan_3], # 3 13 23
    [soft_data_3,hard_voting_2, hard_voting_3, dan_1, dan_2], # 3 23 12
    [soft_data_3,hard_voting_2, hard_voting_3, dan_1, dan_3], # 3 23 13
    [soft_data_3,hard_voting_2, hard_voting_3, dan_2, dan_3], # 3 23 23
]

In [ ]:
#정답에 빈값이 있는경우, 중간에 주석처리로 변경가능
#최빈값 찾기

list_data = [soft_data_2,hard_data_1,hard_data_2, dan_data_2,dan_data_3]
list_wv = [wv_0, wv_1, wv_2, wv_3, wv_4]
list_aaa = []


for i in range(len(soft_data_3[0])):
  board = defaultdict(int)
  
  for j in range(len(list_data)):

    # #공백정답포함
    # annotation = ''
    # for q in range(len(list_data[j][0][i]['annotation'])):
    #   if q == len(list_data[j][0][i]['annotation'])-1:
    #     annotation = annotation + list_data[j][0][i]['annotation'][q][0] +'$'+ list_data[j][0][i]['annotation'][q][1]
    #   else:
    #     annotation = annotation + list_data[j][0][i]['annotation'][q][0] +'$'+ list_data[j][0][i]['annotation'][q][1] +','   
    # board[annotation] += list_wv[j]
   
   #공백정답미포함
    annotation = ''
    for q in range(len(list_data[j][0][i]['annotation'])):
      if q == len(list_data[j][0][i]['annotation'])-1:
        annotation = annotation + list_data[j][0][i]['annotation'][q][0] +'$'+ list_data[j][0][i]['annotation'][q][1]
      else:
        annotation = annotation + list_data[j][0][i]['annotation'][q][0] +'$'+ list_data[j][0][i]['annotation'][q][1] +','   
    if annotation != '':
      board[annotation] += list_wv[j]


  tmp = [k for k,v in board.items() if max(board.values()) == v]
  list_aaa.append(tmp)



for i in range(len(list_aaa)):
  data_a = []
  if len(list_aaa[i]) != 0:
    for j in range(len(list_aaa[i][0].split(','))):
      if list_aaa[i][0] != '':
        data_b = []
        data_b.append(list_aaa[i][0].split(',')[j].split('$')[0])
        data_b.append(list_aaa[i][0].split(',')[j].split('$')[1])
        data_a.append(data_b)
      

    data_final[0][i]['annotation'] = data_a


with open("soft_hard_21223_검증.json", "w") as f:
    json.dump(data_final[0], f)